**the copy based on copy1, to conduct PCA before regression.** <br>
Due to large dimension and computing capacity, must reduce dimension before doing regression.<br>
Last version, failed to run RFE for feature selection. RFE is better when choosing a few features from a relatively small feature space

In [2]:
import sys
sys.path.append(r'C:\Users\lzowe\OneDrive - The City College of New York\CCNY_Course\Applied_Machine_Learning_and_Data_Mining\codes\project-product-price-prediction')

In [3]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from final.feature_extraction.vectorization import text_vectorizaion
from final.dimension_reduction.feature_reduction import dimension_reduction

matplotlib.rcParams['figure.figsize'] = (15, 15)
plt.rcParams['figure.constrained_layout.use'] = True

# Data preparation for regression

## loading data processed using Tokenizing and tf-idf algorithm

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/DSEI21000-S21/project-product-price-prediction/main/data/random_samples/stratified_sampling_clean_text_data_by_price_whigh_sz50000_1619835594.csv")

In [5]:
df.shape

(50000, 34)

In [6]:
df.describe()

,train_id,item_description_bef_word_count,item_description_bef_char_count,item_description_bef_avg_word_len,item_description_upper_word_count,item_description_upper_char_count,item_description_stopword_count,item_description_punctuation_count,item_description_number_count,item_description_after_word_count,...,item_name_upper_char_count,item_name_stopword_count,item_name_punctuation_count,item_name_number_count,item_name_after_word_count,item_name_after_char_count,item_name_after_avg_word_len,item_condition_id,shipping,price
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000
mean,7.373508e+05,26.583180,150.390920,5.667798,1.876040,12.577260,7.67970,5.850200,0.499880,18.323300,...,4.794540,0.192960,0.414340,0.177680,4.236300,24.381740,5.845573,1.991820,0.376700,108.41749
std,4.251891e+05,28.205148,161.300927,0.782677,5.826971,28.114883,10.36772,8.336725,1.229061,18.691275,...,5.226287,0.463088,0.828128,0.422508,1.535705,8.740065,1.210724,0.896911,0.484564,198.75487
min,1.900000e+01,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.00000
25%,3.726685e+05,8.000000,46.000000,5.210526,0.000000,2.000000,1.00000,1.000000,0.000000,7.000000,...,2.000000,0.000000,0.000000,0.000000,3.000000,18.000000,5.000000,1.000000,0.000000,20.00000
50%,7.353620e+05,17.000000,97.000000,5.642857,0.000000,5.000000,4.00000,3.000000,0.000000,12.000000,...,3.000000,0.000000,0.000000,0.000000,4.000000,25.000000,5.750000,2.000000,0.000000,50.00000
75%,1.102881e+06,34.000000,191.000000,6.020944,1.000000,12.000000,10.00000,8.000000,1.000000,23.000000,...,6.000000,0.000000,1.000000,0.000000,5.000000,32.000000,6.500000,3.000000,1.000000,90.00000
max,1.482519e+06,206.000000,1007.000000,19.600000,178.000000,761.000000,104.00000,308.000000,57.000000,175.000000,...,37.000000,6.000000,12.000000,9.000000,13.000000,42.000000,26.000000,5.000000,1.000000,2009.00000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   train_id                             50000 non-null  int64  
 1   clean_item_description               50000 non-null  object 
 2   item_description_bef_word_count      50000 non-null  float64
 3   item_description_bef_char_count      50000 non-null  float64
 4   item_description_bef_avg_word_len    50000 non-null  float64
 5   item_description_upper_word_count    50000 non-null  float64
 6   item_description_upper_char_count    50000 non-null  float64
 7   item_description_stopword_count      50000 non-null  float64
 8   item_description_punctuation_count   50000 non-null  float64
 9   item_description_number_count        50000 non-null  float64
 10  item_description_after_word_count    50000 non-null  float64
 11  item_description_after_char_

In [8]:
df.clean_item_name.unique().shape

(42039,)

In [9]:
df.head()

,train_id,clean_item_description,item_description_bef_word_count,item_description_bef_char_count,item_description_bef_avg_word_len,item_description_upper_word_count,item_description_upper_char_count,item_description_stopword_count,item_description_punctuation_count,item_description_number_count,...,item_name_after_avg_word_len,item_condition_id,category_name,brand_name,shipping,price,c1,c2,c3,price_bin
0,806824,new tags,3.0,13.0,4.333333,0.0,1.0,1.0,0.0,0.0,...,5.250000,1,Women/Athletic Apparel/Shirts & Tops,Nike,1,15.0,women,athletic apparel,shirts & tops,"(10, 15]"
1,772820,nastasya every hills lipstick fashion,6.0,42.0,7.000000,0.0,4.0,1.0,0.0,0.0,...,10.000000,1,Beauty/Makeup/Lips,Anastasia Beverly Hills,0,22.0,beauty,makeup,lips,"(20, 25]"
2,1423115,brand new tags taken bag pictures,11.0,54.0,4.909091,0.0,1.0,5.0,0.0,0.0,...,6.166667,1,Women/Jeans/Leggings,LuLaRoe,0,54.0,women,jeans,leggings,"(50, 60]"
3,405853,bought calves bit large frowned good condition...,35.0,188.0,5.371429,1.0,4.0,17.0,7.0,1.0,...,5.333333,3,Women/Shoes/Boots,Hunter,0,84.0,women,shoes,boots,"(80, 90]"
4,1172086,brand new box size 7youth859womens,6.0,40.0,6.666667,0.0,3.0,1.0,3.0,0.0,...,4.000000,1,Women/Shoes/Athletic,Nike,0,56.0,women,shoes,athletic,"(50, 60]"


 ## Feature extraction and dimension selection
 
 For Item-discription feature: <br>
 using Jin's function to, firstly, do feature-extraction, increasing up to 14230 new few features <br>
 secondly, do dimenstion-reduction <br>
 finally, left 100 

In [10]:
df.clean_item_description[12]

'corners bottom great shape lips smells markings inside cleanthere small water mark indicated third photo comes dusting'

In [11]:
description_feature,  description_feature_name = text_vectorizaion(df, text_col = "clean_item_description", 
                                                                   tfidf = True, min_df=10, max_features=100000)

In [12]:
description_feature.shape

(50000, 14230)

In [13]:
data = dimension_reduction(description_feature.toarray(), method = 'SVD', n_comp = 100)

In [14]:
data.shape

(50000, 100)

In [15]:
df_cid = pd.DataFrame(data.copy()) #df for cleaned item description transforming to new features
df_cid.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000323,0.316579,0.226464,-0.093895,-0.145724,-0.160601,-0.003327,-0.018078,0.090894,0.191935,...,0.002638,0.006700,0.005712,-0.000512,-0.001334,-0.002025,-0.007014,-0.008020,0.009870,0.010357
1,0.000036,0.006363,-0.001485,-0.000307,-0.003073,0.008709,0.000232,-0.001268,-0.002304,0.001756,...,0.001377,0.015823,0.000063,-0.012320,0.029981,-0.011197,0.001984,0.000641,0.009679,-0.009161
2,0.000307,0.282559,0.198734,-0.108194,-0.022312,-0.049010,-0.093214,0.013352,-0.023772,0.027221,...,-0.008231,0.019248,0.017355,0.000174,0.027450,0.014739,0.036572,-0.010762,0.020620,0.026407
3,0.000196,0.087350,-0.138583,-0.028884,-0.007241,-0.041194,-0.030514,0.157935,0.007513,0.001806,...,-0.007530,0.000919,-0.015024,-0.001424,-0.033341,-0.001557,-0.045089,-0.005682,-0.003724,0.000257
4,0.000409,0.407447,0.230953,-0.090665,-0.028016,-0.120192,-0.079092,0.052900,-0.152950,-0.178806,...,0.006526,-0.016268,0.005182,0.002841,0.005126,0.012338,0.012810,0.001393,0.010066,0.024485


## Concatenating new features df_cid and previous df

In [16]:
df1=df.copy()
df1.drop("clean_item_description", inplace=True,axis=1)
df1.head()

,train_id,item_description_bef_word_count,item_description_bef_char_count,item_description_bef_avg_word_len,item_description_upper_word_count,item_description_upper_char_count,item_description_stopword_count,item_description_punctuation_count,item_description_number_count,item_description_after_word_count,...,item_name_after_avg_word_len,item_condition_id,category_name,brand_name,shipping,price,c1,c2,c3,price_bin
0,806824,3.0,13.0,4.333333,0.0,1.0,1.0,0.0,0.0,2.0,...,5.250000,1,Women/Athletic Apparel/Shirts & Tops,Nike,1,15.0,women,athletic apparel,shirts & tops,"(10, 15]"
1,772820,6.0,42.0,7.000000,0.0,4.0,1.0,0.0,0.0,5.0,...,10.000000,1,Beauty/Makeup/Lips,Anastasia Beverly Hills,0,22.0,beauty,makeup,lips,"(20, 25]"
2,1423115,11.0,54.0,4.909091,0.0,1.0,5.0,0.0,0.0,6.0,...,6.166667,1,Women/Jeans/Leggings,LuLaRoe,0,54.0,women,jeans,leggings,"(50, 60]"
3,405853,35.0,188.0,5.371429,1.0,4.0,17.0,7.0,1.0,17.0,...,5.333333,3,Women/Shoes/Boots,Hunter,0,84.0,women,shoes,boots,"(80, 90]"
4,1172086,6.0,40.0,6.666667,0.0,3.0,1.0,3.0,0.0,5.0,...,4.000000,1,Women/Shoes/Athletic,Nike,0,56.0,women,shoes,athletic,"(50, 60]"


In [17]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   train_id                             50000 non-null  int64  
 1   item_description_bef_word_count      50000 non-null  float64
 2   item_description_bef_char_count      50000 non-null  float64
 3   item_description_bef_avg_word_len    50000 non-null  float64
 4   item_description_upper_word_count    50000 non-null  float64
 5   item_description_upper_char_count    50000 non-null  float64
 6   item_description_stopword_count      50000 non-null  float64
 7   item_description_punctuation_count   50000 non-null  float64
 8   item_description_number_count        50000 non-null  float64
 9   item_description_after_word_count    50000 non-null  float64
 10  item_description_after_char_count    50000 non-null  float64
 11  item_description_after_avg_w

In [18]:
df1.shape,df_cid.shape

((50000, 33), (50000, 100))

In [19]:
df_concat=pd.concat([df1,df_cid],axis=1)
df_concat.shape

(50000, 133)

## Using one-hot encoding for category and nominal features

delete price bins

In [20]:
cols_catNnom = ['category_name','brand_name', 'c1', 'c2', 'c3'] # columns of category and nominal features

In [21]:
from final.feature_encoding.one_hot_encoding import one_hot_encode_feature
df_encode = df_concat
for col in cols_catNnom:
    df_encode, col_encode = one_hot_encode_feature(df_encode, encode_column=col,drop_first=False)
df_encode.shape

(50000, 3028)

In [22]:
df_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 3028 entries, train_id to c3_yoga & pilates
dtypes: float64(123), int64(3), object(2), uint8(2900)
memory usage: 187.1+ MB


In [23]:
df_encode.head()

,train_id,item_description_bef_word_count,item_description_bef_char_count,item_description_bef_avg_word_len,item_description_upper_word_count,item_description_upper_char_count,item_description_stopword_count,item_description_punctuation_count,item_description_number_count,item_description_after_word_count,...,c3_window treatments,"c3_wine, beer & beverage coolers",c3_wipes & holders,c3_women,c3_women's golf clubs,c3_wool,c3_work & safety,c3_wrap,c3_writing,c3_yoga & pilates
0,806824,3.0,13.0,4.333333,0.0,1.0,1.0,0.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,772820,6.0,42.0,7.000000,0.0,4.0,1.0,0.0,0.0,5.0,...,0,0,0,0,0,0,0,0,0,0
2,1423115,11.0,54.0,4.909091,0.0,1.0,5.0,0.0,0.0,6.0,...,0,0,0,0,0,0,0,0,0,0
3,405853,35.0,188.0,5.371429,1.0,4.0,17.0,7.0,1.0,17.0,...,0,0,0,0,0,0,0,0,0,0
4,1172086,6.0,40.0,6.666667,0.0,3.0,1.0,3.0,0.0,5.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_encode[["price"]]

,price
0,15.0
1,22.0
2,54.0
3,84.0
4,56.0
...,...
49995,1609.0
49996,205.0
49997,36.0
49998,20.0


In [25]:
# prepare input X, y
X, y = df_encode.copy().drop(["clean_item_name","train_id","price","price_bin"],axis=1), df_encode[["price"]] 

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 3024 entries, item_description_bef_word_count to c3_yoga & pilates
dtypes: float64(122), int64(2), uint8(2900)
memory usage: 185.6 MB


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=42)

## Conduct dimension reduction on the dateframe processed by one-hot encoding
skip this step in the first place. <br>
let us see how the regression result looks like. and then determine if the dimension reduction on whole dataset needed, or if use other techniques to avoid overfitting(e.g. adding penalty, using RobustScaler, conducting cross validation).<br>



* log 1: actually can't skip this step, the dimension is too large to run, Exception was raised: MemoryError: Unable to allocate 11.8 GiB for an array with shape (45079, 35000) and data type float64.
Therefore, try conduct PCA again before regression

* log 2: even can't not run PCA due to the large number of features. Try drop clean_item_name instead of encoding it, as it is has very less duplicated items.

* log 3: Since log2 works, skip this step( e.i. 1.5) for now.

* log 4: features are too much later, have to reduce dimension at this step.

In [32]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [41]:
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
pca.explained_variance_ratio_

array([0.00290486, 0.0022789 , 0.00214285, 0.00197305, 0.00188288,
       0.00186249, 0.00180662, 0.00178066, 0.00174815, 0.0016754 ,
       0.00162334, 0.0016063 , 0.00155175, 0.00154981, 0.00147137,
       0.00146217, 0.00145709, 0.00145545, 0.0014492 , 0.00144028,
       0.00143402, 0.00142974, 0.00142609, 0.00142109, 0.00141807,
       0.00141246, 0.00141036, 0.00140121, 0.00139544, 0.00139292,
       0.0013886 , 0.00138174, 0.00138119, 0.00136717, 0.00136214,
       0.00135612, 0.00134181, 0.00133579, 0.00132456, 0.0013219 ,
       0.00130613, 0.00130098, 0.0012937 , 0.00129239, 0.00128503,
       0.00128127, 0.00127723, 0.00126964, 0.0012677 , 0.00126545,
       0.00125469, 0.0012481 , 0.00124392, 0.00123624, 0.00121776,
       0.00120707, 0.00120143, 0.00119584, 0.00119393, 0.00119201,
       0.00118449, 0.00117942, 0.00117763, 0.00116285, 0.00115652,
       0.00115441, 0.00115194, 0.00115066, 0.0011489 , 0.00113124,
       0.00112673, 0.00112621, 0.00112206, 0.00111204, 0.00110

In [42]:
X_train_pca.shape, X_test_pca.shape 

((35000, 500), (15000, 500))

# Regression Model Selection
Proposal:
1. try different regression techniques, and select the best one;
      - DT regressor
      * Ensemble method
      - Adding polynomial items to multiple linear regression? 
      - Tensorflow.Keras
2. focus on the selected one and tuning modelling.(Guessing propably would use Ensemble learning or Keras in the end)<br>
**Based on the our previous experiment so far, our results have both large bias and variance.** Probably need to use more powerful algorithm to improve the bias, and then considering solve overfitting issue?

## Regression Tree

In [44]:
from sklearn import tree
from sklearn.metrics import mean_squared_error, explained_variance_score,r2_score 
def evaluate_dt_regressor(X_train, X_test, y_train, y_test,n_iter,max_depth):
    # the maximum depth of the tree. If None, then nodes are expanded until all leaves are pure
    MSE_test=[]
    EVS_test=[]
    R2_test=[]
    MSE_train=[]
    EVS_train=[]
    R2_train=[]
    for i in range(n_iter):
        rg = tree.DecisionTreeRegressor(max_depth=max_depth)
        rg.fit(X_train, y_train)
        y_train_pred = rg.predict(X_train)
        y_pred = rg.predict(X_test)
        MSE_train.append(mean_squared_error(y_train, y_train_pred))
        EVS_train.append(explained_variance_score(y_train, y_train_pred))
        R2_train.append(r2_score(y_train, y_train_pred))
        
        MSE_test.append(mean_squared_error(y_test, y_pred))
        EVS_test.append(explained_variance_score(y_test, y_pred))
        R2_test.append(r2_score(y_test, y_pred))
    print("Train score:","MSE: {}".format(sum(MSE_train)/n_iter),"EVS: {}".format(sum(EVS_train)/n_iter),"R2: {}".format(sum(R2_train)/n_iter),)
    print("Test score:","MSE: {}".format(sum(MSE_test)/n_iter),"EVS: {}".format(sum(EVS_test)/n_iter),"R2: {}".format(sum(R2_test)/n_iter),)

In [45]:
evaluate_dt_regressor(X_train_pca, X_test_pca, y_train, y_test,1,max_depth=None)

Train score: MSE: 1.4285714285714285e-05 EVS: 0.9999999996453222 R2: 0.9999999996453222
Test score: MSE: 20444.605316666668 EVS: 0.4576498018365237 R2: 0.45757297223371407


In [47]:
evaluate_dt_regressor(X_train_pca, X_test_pca,y_train, y_test,1,max_depth=3)

Train score: MSE: 27414.984394562318 EVS: 0.3193558770393383 R2: 0.3193558770393383
Test score: MSE: 24911.631535077548 EVS: 0.3390785961873015 R2: 0.3390558516008496


In [48]:
evaluate_dt_regressor(X_train_pca, X_test_pca,  y_train, y_test,1,max_depth=5)

Train score: MSE: 21470.693098680924 EVS: 0.46693746517660495 R2: 0.46693746517660484
Test score: MSE: 21338.688174806877 EVS: 0.4338543263328638 R2: 0.4338515699466069


In [49]:
evaluate_dt_regressor(X_train_pca, X_test_pca,  y_train, y_test,10,max_depth=None)

Train score: MSE: 1.4285714285714289e-05 EVS: 0.9999999996453222 R2: 0.9999999996453222
Test score: MSE: 20796.324884999998 EVS: 0.4483532295694417 R2: 0.4482413076159205


## Linear Regression

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train_pca, y_train)
reg.coef_

why linear regression is so slow. after 1 and harf hour, not completed

In [66]:
from sklearn.metrics import mean_squared_error, explained_variance_score,r2_score 

In [75]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train_pca, y_train)
y_train_pred = rg.predict(X_train_pca)
y_pred = reg.predict(X_test_pca)

print("train:",
{
    "MSE": mean_squared_error(y_train, y_train_pred), 
    "EVS": explained_variance_score(y_train, y_train_pred), 
    "R2": r2_score(y_train, y_train_pred)
})

print("test:",
{
    "MSE": mean_squared_error(y_test, y_pred), 
    "EVS": explained_variance_score(y_test, y_pred), 
    "R2": r2_score(y_test, y_pred)
})


train: {'MSE': 990.2144264683332, 'EVS': 0.9762126689873514, 'R2': 0.9754155019697835}
test: {'MSE': 22210.446474589295, 'EVS': 0.4107919615689941, 'R2': 0.4107224727516572}


In [64]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train_pca, y_train)
reg.coef_

array([[ 2.01661616e+01,  3.57170497e+00, -1.21743025e+01,
         1.03850638e-01, -5.37371805e+00,  1.19993145e+01,
        -8.29245020e-01,  6.77126180e+00, -2.36707161e+01,
        -6.22912942e+00,  3.10004496e+00, -8.60990629e+00,
         1.39351526e+00, -2.92582226e+00, -5.66847894e+00,
        -4.99938439e-02,  2.48445259e-01,  4.03682124e+00,
        -2.77682057e-03,  1.11183677e+00, -1.30465959e+00,
         5.06837600e+00,  2.08765704e+00, -2.11372111e+00,
         1.46605907e+01,  4.56182235e-01,  2.85324042e+00,
        -1.33500733e+00,  1.53469833e+00, -4.18507276e+00,
        -7.30598243e+00, -3.74429088e+00,  3.58974449e+00,
        -6.73590742e+00,  1.25521492e+01,  3.13108578e+00,
        -1.92969608e+00, -3.13510119e+00,  1.27897515e+00,
        -3.28727765e-02, -8.42735875e-01, -6.53110713e+00,
         4.47492400e+00, -1.39037208e+00, -7.69727479e-01,
         4.62334193e-01, -4.22307302e+00, -2.02309253e+00,
        -2.64448708e+00,  9.42640916e+00,  8.84463204e+0

## Ensemble learning

### Random Forest (Bagging) for classification, also for regression

In [54]:
from sklearn.ensemble import RandomForestRegressor
def evaluate_rf_regressor(X_train, X_test, y_train, y_test,n_iter,max_depth,random_state):
    MSE_test=[]
    EVS_test=[]
    R2_test=[]
    MSE_train=[]
    EVS_train=[]
    R2_train=[]
    for i in range(n_iter):
        rg = RandomForestRegressor(max_depth=max_depth, random_state=random_state)
        rg.fit(X_train, y_train)
        y_train_pred = rg.predict(X_train)
        y_pred = rg.predict(X_test)
        MSE_train.append(mean_squared_error(y_train, y_train_pred))
        EVS_train.append(explained_variance_score(y_train, y_train_pred))
        R2_train.append(r2_score(y_train, y_train_pred))
        
        MSE_test.append(mean_squared_error(y_test, y_pred))
        EVS_test.append(explained_variance_score(y_test, y_pred))
        R2_test.append(r2_score(y_test, y_pred))
    print("Train score:","MSE: {}".format(sum(MSE_train)/n_iter),"EVS: {}".format(sum(EVS_train)/n_iter),"R2: {}".format(sum(R2_train)/n_iter),)
    print("Test score:","MSE: {}".format(sum(MSE_test)/n_iter),"EVS: {}".format(sum(EVS_test)/n_iter),"R2: {}".format(sum(R2_test)/n_iter),)

rf_regressor turns out very very slow

In [55]:
evaluate_rf_regressor(X_train_pca, X_test_pca, y_train, y_test,1,max_depth=None,random_state=42)

<ipython-input-54-ffee7f25d399>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rg.fit(X_train, y_train)


Train score: MSE: 990.2144264683332 EVS: 0.9762126689873514 R2: 0.9754155019697835
Test score: MSE: 8995.427674365284 EVS: 0.7621419387473207 R2: 0.7613373786719745


#### Check Feature Importance

In [56]:
rg = RandomForestRegressor(max_depth=None, random_state=42)
rg.fit(X_train_pca, y_train)
y_train_pred = rg.predict(X_train_pca)
y_pred = rg.predict(X_test_pca)
MSE_train.append(mean_squared_error(y_train, y_train_pred))
EVS_train.append(explained_variance_score(y_train, y_train_pred))
R2_train.append(r2_score(y_train, y_train_pred))

MSE_test.append(mean_squared_error(y_test, y_pred))
EVS_test.append(explained_variance_score(y_test, y_pred))
R2_test.append(r2_score(y_test, y_pred))
print("Train score:","MSE: {}".format(sum(MSE_train)/n_iter),"EVS: {}".format(sum(EVS_train)/n_iter),"R2: {}".format(sum(R2_train)/n_iter),)
print("Test score:","MSE: {}".format(sum(MSE_test)/n_iter),"EVS: {}".format(sum(EVS_test)/n_iter),"R2: {}".format(sum(R2_test)/n_iter),)

<ipython-input-56-558030b727bd>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rg.fit(X_train_pca, y_train)


NameError: name 'MSE_train' is not defined

In [57]:
importances = rg.feature_importances_
importances.shape

(500,)

In [58]:
X.shape

(50000, 3024)

In [62]:
feat_labels = range(X_train_pca.shape[1])
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importance')
plt.bar(range(X_train_pca.shape[1]), 
        importances[indices],
        align='center')

plt.xticks(range(X_train_pca.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train_pca.shape[1]])
plt.tight_layout()
#plt.savefig('images/04_09.png', dpi=300)
plt.show()

 1) 7                              0.105877
 2) 24                             0.063736
 3) 33                             0.055135
 4) 288                            0.033101
 5) 1                              0.030501
 6) 10                             0.025732
 7) 316                            0.017765
 8) 89                             0.014740
 9) 48                             0.012854
10) 469                            0.012297
11) 11                             0.010544
12) 262                            0.009389
13) 2                              0.008283
14) 0                              0.007948
15) 14                             0.007850
16) 491                            0.007349
17) 57                             0.006804
18) 6                              0.006769
19) 34                             0.006605
20) 46                             0.006498
21) 261                            0.005871
22) 476                            0.005158
23) 483                         

IndexError: index 500 is out of bounds for axis 0 with size 500

### Boosting

### Stacking 

## Turning multiple linear regression with polynomial regression

## Tensorflow

In [70]:
import tensorflow as tf
from tensorflow import keras

In [71]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="sigmoid"))
model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="sigmoid"))

# Fine tune selected model

## Cross_validation

## validation curve vs learning curve

## Regularization penalty

# make it pipeline and apply to the whole dataset